# Tuning Sklearn Models with Ray

Using the Santander Dataset from: 
https://www.kaggle.com/c/santander-customer-transaction-prediction/
This dataset is about predicting customer transactions based on obfuscated features

In [1]:
import pandas as pd

In [2]:
#load train
train = pd.read_csv("data/train.csv")

In [3]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [19]:
train.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

In [15]:
subset_train = train.iloc[:, 1:30]
subset_train.shape

(200000, 29)

In [16]:
train_sample = subset_train.sample(n=20000)

## Quick EDA

quick eda on the dataset just to see if there is any particular "weirdness"

In [6]:
from pandas_profiling import ProfileReport

In [17]:
profile = train_sample.profile_report(
    html = {
        "inline": False
    }
)
profile.to_file("test_report.html")

Test Widgets

In [9]:
profile.to_widgets()

,df_index,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17
0,148208,0,8.7637,-3.4904,8.9789,7.6137,13.0118,-5.0564,5.9660,14.9309,4.4759,5.5201,-10.2331,3.5918,14.0316,14.3582,5.4025,14.5775,11.8943,-13.3084
1,146652,0,12.1435,-3.7215,13.3433,4.3677,12.9936,-12.9412,6.6048,22.4650,-0.8970,7.4700,-2.5102,-5.7761,14.2124,10.0028,9.6739,14.3188,13.2056,-8.1248
2,141438,0,8.8203,1.7712,9.8879,7.7618,10.5102,-8.7845,5.4810,16.8766,-1.7048,6.5894,9.0411,-10.2293,14.2251,8.0387,10.1773,14.6263,8.4790,-7.7529
3,69546,0,14.2036,2.4195,11.1785,6.6001,8.7996,-2.6245,4.9558,10.4340,3.0301,6.0642,8.5782,-7.6497,14.3283,6.9095,5.8546,13.9954,15.7102,-8.6177
4,136835,0,14.3887,-5.8229,14.0484,7.7986,10.5184,-8.6272,5.7404,19.6747,-3.2407,6.9349,-8.3565,6.7346,14.2269,11.2714,9.3898,14.3120,8.7654,-7.2079
5,75020,0,10.0044,-3.4247,15.6993,9.6108,12.4087,-6.6813,5.9675,19.9534,-3.1204,5.0346,-5.4289,-1.9962,14.1246,-0.5943,6.2630,14.9752,9.4954,1.4205
6,61643,0,12.3888,-1.1854,8.6229,8.1160,11.2580,-4.5191,5.4660,19.0275,2.3603,8.3663,10.1702,-8.8783,14.0093,4.5120,8.5551,15.1809,11.4872,-14.9605
7,322,0,12.5130,-7.3175,9.4896,7.2075,13.3605,-4.1300,3.6747,16.3560,1.2980,7.4382,6.0542,5.4442,14.0454,10.1226,6.9883,14.3448,11.3432,0.0792
8,165983,0,11.7240,7.8786,9.2545,4.2673,13.9630,4.2373,5.7301,18.1619,2.6006,7.0754,0.8390,0.4139,13.8353,8.8781,8.4607,13.9120,7.5538,-2.0053
9,139502,0,7.2516,-8.3607,11.3766,4.5535,7.6246,-10.9828,5.7700,17.5080,1.0709,9.1434,5.8422,8.5205,14.1978,9.5074,10.0263,14.8440,13.3140,-9.9558


,df_index,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17
19990,16404,0,10.9473,-3.9487,6.0893,8.4611,7.9582,-18.6406,3.9500,18.8571,0.1805,6.9380,-5.3923,-4.4705,13.5762,9.8821,5.8796,15.0879,7.1685,-1.6195
19991,109435,0,8.4275,1.0271,9.0005,8.3209,11.1656,-11.4810,4.0678,19.7957,-4.0674,5.8042,7.5083,-9.9937,13.9720,14.1800,7.4110,14.2729,5.3115,-5.7972
19992,8985,0,16.7690,-5.9631,8.4378,4.2550,10.2804,-9.6719,4.4875,15.6687,-2.7517,6.3474,14.1333,11.0949,13.7502,5.4092,4.0765,13.8644,11.7285,-10.1371
19993,197963,0,8.3431,-0.1214,10.9137,9.1274,12.0479,-12.7615,4.8381,14.4572,-2.5789,6.5039,3.8787,-10.3857,14.1892,3.0632,6.3148,15.1083,8.9413,-2.1972
19994,92464,1,12.6697,0.8551,7.5576,5.5062,11.0614,-22.9457,7.5999,20.6649,-3.0653,6.2146,0.9867,-8.8554,13.8248,6.6384,11.5655,14.8856,10.6554,-17.0570
19995,73278,0,11.1892,-3.7039,7.8432,9.8900,9.3028,-4.0141,5.1761,13.4338,-4.1509,7.7468,-7.9663,-5.6762,14.1313,13.9600,8.0250,15.0623,11.4344,-15.4643
19996,29387,0,9.9973,0.2640,10.4446,5.0784,8.2907,-11.8575,3.8776,14.3387,1.5762,7.7294,-7.2082,4.2611,13.8605,10.9031,6.2187,14.0506,10.9113,-4.9976
19997,89316,1,16.2264,-3.4747,8.3434,10.5620,11.8099,4.0179,5.7363,19.2138,0.8684,6.3900,2.1940,-2.6882,13.9452,18.0352,10.3704,14.9906,12.9073,-6.0311
19998,61332,0,8.9096,-1.8137,7.9458,10.2041,11.9562,-9.2195,4.4841,21.2636,-3.3675,8.6736,-2.1671,-9.4064,14.1034,5.2927,10.2364,13.8241,7.0283,6.4880
19999,179397,0,6.5149,-7.2614,5.6223,9.0028,13.7480,0.0522,5.0912,17.8117,3.3876,7.5577,1.1687,-7.8040,13.7454,5.9868,4.6010,14.5428,9.8654,4.6243


In [10]:
report = train_sample.profile_report()

## Feature Building

testing out feature tools

In [18]:
import featuretools as ft

In [23]:
# extract out the target var from train
target = train['target']
ft_train = train.drop(['target'], axis=1)

In [21]:
es = ft.EntitySet('customer_records')

In [31]:
es.entity_from_dataframe(dataframe=train,
                        entity_id='target',
                        index='ID_code')

Entityset: customer_records
  Entities:
    ID_code [Rows: 200000, Columns: 201]
    target [Rows: 200000, Columns: 202]
  Relationships:
    No relationships

In [32]:
fm, features = ft.dfs(entityset=es, 
                      target_entity='target',
                      max_depth=2,
                      verbose=1)

Built 201 features
Elapsed: 00:03 | Progress: 100%|██████████


In [33]:
features

[<Feature: target>,
 <Feature: var_0>,
 <Feature: var_1>,
 <Feature: var_2>,
 <Feature: var_3>,
 <Feature: var_4>,
 <Feature: var_5>,
 <Feature: var_6>,
 <Feature: var_7>,
 <Feature: var_8>,
 <Feature: var_9>,
 <Feature: var_10>,
 <Feature: var_11>,
 <Feature: var_12>,
 <Feature: var_13>,
 <Feature: var_14>,
 <Feature: var_15>,
 <Feature: var_16>,
 <Feature: var_17>,
 <Feature: var_18>,
 <Feature: var_19>,
 <Feature: var_20>,
 <Feature: var_21>,
 <Feature: var_22>,
 <Feature: var_23>,
 <Feature: var_24>,
 <Feature: var_25>,
 <Feature: var_26>,
 <Feature: var_27>,
 <Feature: var_28>,
 <Feature: var_29>,
 <Feature: var_30>,
 <Feature: var_31>,
 <Feature: var_32>,
 <Feature: var_33>,
 <Feature: var_34>,
 <Feature: var_35>,
 <Feature: var_36>,
 <Feature: var_37>,
 <Feature: var_38>,
 <Feature: var_39>,
 <Feature: var_40>,
 <Feature: var_41>,
 <Feature: var_42>,
 <Feature: var_43>,
 <Feature: var_44>,
 <Feature: var_45>,
 <Feature: var_46>,
 <Feature: var_47>,
 <Feature: var_48>,
 <Feature:

## Building Model

- simple baseline model first

# Standard Fit

Standard fit without any tuning

In [ ]:
# preproc things
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

# Load a Model

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

In [ ]:
score = clf.score(X_test, y_test)
score

# Test Iris

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

In [ ]:
iris = load_iris()
# need to reformat the data
import pandas as pd
train = pd.DataFrame(iris.data)
train.head()

In [ ]:
X_iris_train, X_iris_test, y_iris_train, y_iris_test = \
        train_test_split(train, iris.target, test_size=.4, random_state=42)

In [ ]:
X_iris_train

In [ ]:
y_iris_test.shape

In [ ]:
clf_iris = tree.DecisionTreeClassifier()
clf_iris.fit(X_iris_train, y_iris_train)

In [ ]:
y_test_result = clf_iris.predict(X_iris_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:
confusion_matrix(y_test_result, y_iris_test)

In [ ]:
class_names = iris.target_names

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plot_confusion_matrix(clf_iris, X_iris_test, y_iris_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=None)